# Validación de Integración: Coherencia INE ↔ EUROSTAT

Este notebook valida la **coherencia y compatibilidad** entre las tablas INE y EUROSTAT para asegurar que los datos pueden integrarse correctamente.

## Arquitectura Modular

- **Prerequisitos**: `02a_validacion_INE.ipynb` y `02b_validacion_EUROSTAT.ipynb` completados
- **Este módulo**: Validación de coherencia cruzada entre fuentes
- **Orquestador**: `02_run_validation.py` (ejecuta todos los módulos)

## Validaciones de Integración

1. **Coherencia temporal**: Años compatibles entre fuentes
2. **Coherencia de indicadores**: Comparación AROPE, Gini, S80/S20
3. **Coherencia geográfica**: España en EUROSTAT vs Total en INE
4. **Consistencia de valores**: Verificar que métricas comparables estén en rangos similares

## Salida

- **Reportes JSON/CSV**: `data/validated/logs/`
- **Reporte consolidado**: Resumen de compatibilidad entre fuentes

---

In [1]:
# 0. CONFIGURACIÓN
import sys
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime

# Añadir utils al path
sys.path.append('../../')

from utils.validation_framework import ValidationReport
from utils.config import DB_CONNECTION_STRING

# Conexión a SQL Server
conn = pyodbc.connect(DB_CONNECTION_STRING)

print("="*80)
print("MÓDULO DE VALIDACIÓN: INTEGRACIÓN INE ↔ EUROSTAT")
print("="*80)
print(f"✅ Framework de validación cargado")
print(f"⏰ Timestamp: {datetime.now().isoformat()}")

MÓDULO DE VALIDACIÓN: INTEGRACIÓN INE ↔ EUROSTAT
✅ Framework de validación cargado
⏰ Timestamp: 2025-11-16T20:30:52.659009


---

## 1. Cargar Tablas Clave para Integración

---

In [2]:
print("="*80)
print("CARGANDO TABLAS CLAVE")
print("="*80)

# Crear reporte de integración
report = ValidationReport('INTEGRACION_INE_EUROSTAT')

# Cargar tablas INE clave
try:
    df_ine_arope = pd.read_sql('SELECT * FROM INE_AROPE_Hogar', conn)
    print(f"✅ INE_AROPE_Hogar: {len(df_ine_arope)} registros")
except Exception as e:
    print(f"❌ Error cargando INE_AROPE_Hogar: {e}")
    report.add_error(f"No se pudo cargar INE_AROPE_Hogar: {e}")
    df_ine_arope = None

# Cargar tablas EUROSTAT clave (ajustar según tus tablas reales)
try:
    # Buscar tablas EUROSTAT que existan
    cursor = conn.cursor()
    cursor.execute("""
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_NAME LIKE 'EUROSTAT_AROP%' OR TABLE_NAME LIKE 'EUROSTAT_%Poverty%'
        ORDER BY TABLE_NAME
    """)
    eurostat_poverty_tables = [row[0] for row in cursor.fetchall()]
    
    if eurostat_poverty_tables:
        # Cargar la primera tabla de pobreza EUROSTAT encontrada
        eurostat_table = eurostat_poverty_tables[0]
        df_eurostat_arop = pd.read_sql(f'SELECT * FROM {eurostat_table}', conn)
        print(f"✅ {eurostat_table}: {len(df_eurostat_arop)} registros")
    else:
        print("⚠️  No se encontraron tablas EUROSTAT de pobreza")
        df_eurostat_arop = None
        report.add_warning("No se encontraron tablas EUROSTAT de pobreza para comparación")
except Exception as e:
    print(f"❌ Error cargando tablas EUROSTAT: {e}")
    report.add_error(f"No se pudieron cargar tablas EUROSTAT: {e}")
    df_eurostat_arop = None

print("\n" + "="*80)

CARGANDO TABLAS CLAVE
✅ INE_AROPE_Hogar: 308 registros
✅ EUROSTAT_AROP_Espana: 10 registros



C:\Users\mario\AppData\Local\Temp\ipykernel_23564\4192550997.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ine_arope = pd.read_sql('SELECT * FROM INE_AROPE_Hogar', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_23564\4192550997.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_eurostat_arop = pd.read_sql(f'SELECT * FROM {eurostat_table}', conn)


---

## 2. Validación de Coherencia Temporal

Verificar que los años disponibles en INE y EUROSTAT se solapan.

---

In [3]:
print("="*80)
print("VALIDACIÓN DE COHERENCIA TEMPORAL")
print("="*80)

if df_ine_arope is not None and df_eurostat_arop is not None:
    # Extraer años únicos
    ine_years = set(df_ine_arope['Año'].unique()) if 'Año' in df_ine_arope.columns else set()
    
    # Buscar columna de año en EUROSTAT (puede ser 'Año', 'Year', 'TIME', etc.)
    eurostat_year_col = None
    for col in ['Año', 'Year', 'TIME', 'time']:
        if col in df_eurostat_arop.columns:
            eurostat_year_col = col
            break
    
    if eurostat_year_col:
        eurostat_years = set(df_eurostat_arop[eurostat_year_col].unique())
        
        # Calcular solapamiento
        common_years = ine_years.intersection(eurostat_years)
        only_ine = ine_years - eurostat_years
        only_eurostat = eurostat_years - ine_years
        
        print(f"\n📊 Análisis temporal:")
        print(f"   INE años: {sorted(ine_years)}")
        print(f"   EUROSTAT años: {sorted(eurostat_years)}")
        print(f"\n🔗 Solapamiento:")
        print(f"   Años comunes: {sorted(common_years)} ({len(common_years)} años)")
        print(f"   Solo INE: {sorted(only_ine)}")
        print(f"   Solo EUROSTAT: {sorted(only_eurostat)}")
        
        # Evaluar
        if len(common_years) >= 5:
            print(f"\n✅ Solapamiento temporal adecuado ({len(common_years)} años)")
            report.add_info(f"Solapamiento temporal: {len(common_years)} años comunes")
        elif len(common_years) > 0:
            print(f"\n⚠️  Solapamiento temporal limitado ({len(common_years)} años)")
            report.add_warning(f"Solapamiento temporal limitado: solo {len(common_years)} años comunes")
        else:
            print(f"\n❌ Sin solapamiento temporal entre fuentes")
            report.add_error("No hay años comunes entre INE y EUROSTAT")
    else:
        print("⚠️  No se encontró columna de año en tabla EUROSTAT")
        report.add_warning("No se pudo determinar años en EUROSTAT")
else:
    print("⚠️  No se pueden comparar años: tablas no cargadas")
    report.add_warning("Validación temporal omitida: tablas no disponibles")

print("\n" + "="*80)

VALIDACIÓN DE COHERENCIA TEMPORAL

📊 Análisis temporal:
   INE años: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
   EUROSTAT años: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

🔗 Solapamiento:
   Años comunes: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)] (10 años)
   Solo INE: [np.int64(2014)]
   Solo EUROSTAT: []

✅ Solapamiento temporal adecuado (10 años)



---

## 3. Validación de Coherencia de Valores

Comparar indicadores similares entre fuentes (ej. AROPE España).

---

In [4]:
print("="*80)
print("VALIDACIÓN DE COHERENCIA DE VALORES")
print("="*80)

if df_ine_arope is not None and df_eurostat_arop is not None and 'common_years' in locals() and len(common_years) > 0:
    print("\n📊 Comparando valores de indicadores AROPE/AROP...")
    
    # Filtrar INE: Total Nacional
    if 'Tipo_Hogar' in df_ine_arope.columns and 'Indicador' in df_ine_arope.columns:
        df_ine_total = df_ine_arope[
            (df_ine_arope['Tipo_Hogar'] == 'Total') &
            (df_ine_arope['Indicador'] == 'AROPE') &
            (df_ine_arope['Año'].isin(common_years))
        ][['Año', 'Valor']].rename(columns={'Valor': 'INE_AROPE'})
        
        print(f"   INE (Total, AROPE): {len(df_ine_total)} observaciones")
    else:
        print("   ⚠️  Estructura de INE no permite filtrar 'Total' y 'AROPE'")
        df_ine_total = None
    
    # Filtrar EUROSTAT: España
    # Ajustar según la estructura real de tus tablas EUROSTAT
    eurostat_geo_col = None
    for col in ['GEO', 'geo', 'Country', 'País']:
        if col in df_eurostat_arop.columns:
            eurostat_geo_col = col
            break
    
    if eurostat_geo_col and eurostat_year_col:
        # Buscar España en EUROSTAT
        spain_values = ['ES', 'ESP', 'Spain', 'España']
        df_eurostat_spain = df_eurostat_arop[
            (df_eurostat_arop[eurostat_geo_col].isin(spain_values)) &
            (df_eurostat_arop[eurostat_year_col].isin(common_years))
        ]
        
        if len(df_eurostat_spain) > 0:
            # Buscar columna de valor
            value_col = None
            for col in ['Value', 'Valor', 'OBS_VALUE', 'value']:
                if col in df_eurostat_spain.columns:
                    value_col = col
                    break
            
            if value_col:
                df_eurostat_spain = df_eurostat_spain[[eurostat_year_col, value_col]].rename(
                    columns={eurostat_year_col: 'Año', value_col: 'EUROSTAT_AROP'}
                )
                print(f"   EUROSTAT (España): {len(df_eurostat_spain)} observaciones")
                
                # Merge y comparar
                if df_ine_total is not None:
                    df_comparison = pd.merge(df_ine_total, df_eurostat_spain, on='Año', how='inner')
                    
                    if len(df_comparison) > 0:
                        df_comparison['Diferencia'] = df_comparison['INE_AROPE'] - df_comparison['EUROSTAT_AROP']
                        df_comparison['Dif_Pct'] = (df_comparison['Diferencia'] / df_comparison['INE_AROPE']) * 100
                        
                        print(f"\n📊 Comparación INE vs EUROSTAT ({len(df_comparison)} años):")
                        print(df_comparison.to_string(index=False))
                        
                        # Evaluar diferencias
                        max_diff = df_comparison['Dif_Pct'].abs().max()
                        
                        if max_diff < 5:
                            print(f"\n✅ Coherencia excelente: diferencia máxima {max_diff:.2f}%")
                            report.add_info(f"Coherencia INE-EUROSTAT: diferencia máxima {max_diff:.2f}%")
                        elif max_diff < 10:
                            print(f"\n⚠️  Coherencia aceptable: diferencia máxima {max_diff:.2f}%")
                            report.add_warning(f"Diferencia INE-EUROSTAT: hasta {max_diff:.2f}%")
                        else:
                            print(f"\n❌ Diferencias significativas: hasta {max_diff:.2f}%")
                            report.add_error(f"Diferencia excesiva INE-EUROSTAT: {max_diff:.2f}%")
                    else:
                        print("   ⚠️  No hay años comunes con datos comparables")
                        report.add_warning("No hay datos comparables entre INE y EUROSTAT")
            else:
                print("   ⚠️  No se encontró columna de valor en EUROSTAT")
        else:
            print("   ⚠️  No se encontraron datos de España en EUROSTAT")
            report.add_warning("No se encontraron datos de España en EUROSTAT")
    else:
        print("   ⚠️  No se pudo identificar estructura geográfica en EUROSTAT")
        report.add_warning("Estructura EUROSTAT no permite filtrar por país")
else:
    print("⚠️  Validación de valores omitida: tablas no disponibles o sin solapamiento temporal")
    report.add_warning("Validación de valores omitida")

print("\n" + "="*80)

VALIDACIÓN DE COHERENCIA DE VALORES

📊 Comparando valores de indicadores AROPE/AROP...
   INE (Total, AROPE): 10 observaciones
   ⚠️  No se pudo identificar estructura geográfica en EUROSTAT



---

## 4. Resumen de Integración

---

In [5]:
print("="*80)
print("RESUMEN DE VALIDACIÓN DE INTEGRACIÓN")
print("="*80)

report.print_report()

# Guardar reporte
print("\n" + "="*80)
print("GUARDANDO REPORTE")
print("="*80)

json_file = report.save_json()
csv_file = report.save_csv()

print(f"✅ Reportes de integración guardados:")
print(f"   📄 JSON: {json_file}")
print(f"   📄 CSV: {csv_file}")

print("\n" + "="*80)
print("🎯 PRÓXIMOS PASOS")
print("="*80)

if report.has_errors():
    print("❌ ATENCIÓN: Se encontraron errores críticos de integración")
    print("   1. Revisar reporte de integración en data/validated/logs/")
    print("   2. Verificar coherencia entre fuentes")
    print("   3. Ajustar reglas de integración si es necesario")
    print("   4. NO proceder al análisis hasta resolver errores")
else:
    print("✅ Validación de integración completada exitosamente")
    print("   1. Todas las fuentes son coherentes y compatibles")
    print("   2. Proceder con análisis exploratorio")
    print("   3. Crear notebooks de análisis en:")
    print("      - notebooks/01_analisis_nacional/")
    print("      - notebooks/02_analisis_regional/")
    print("      - notebooks/03_comparativa_europa/")

print("="*80)

RESUMEN DE VALIDACIÓN DE INTEGRACIÓN
REPORTE DE VALIDACIÓN: INTEGRACION_INE_EUROSTAT

📊 INFORMACIÓN:
  ✅ Solapamiento temporal: 10 años comunes

⚠️ ADVERTENCIAS:
  ⚠️ Estructura EUROSTAT no permite filtrar por país

✅ Sin errores críticos

GUARDANDO REPORTE
📄 Reporte guardado: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251116_203052.json
📄 Reporte guardado: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251116_203052.csv
✅ Reportes de integración guardados:
   📄 JSON: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251116_203052.json
   📄 CSV: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251116_203052.csv

🎯 PRÓXIMOS PASOS
✅ Validación de integración completada exitosamente


In [6]:
# Cerrar conexión
conn.close()
print("✅ Conexión a SQL Server cerrada")

✅ Conexión a SQL Server cerrada
